<table style="margin-left: auto; margin-right: auto; width: 1000px;">
<tbody>
<tr>
<td style="width: 1000px; text-align: left; vertical-align: middle;">
<h1 style="text-align: left;"><strong>Shipwreck Detection</strong></h1>
<h4 style="text-align: left;">Using the ArcGIS API for Python Deep Learning&nbsp;</h4>
</td>
<td style="width: 500px;"><img style="display: block; margin-left: auto; margin-right: auto;" src="https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/tobermory-1572973894.jpg?crop=0.665xw:1.00xh;0.130xw,0&amp;resize=540:*" alt="" width="500" height=100% /></td>
</tr>
</tbody>
</table>

## Abandoned and Derelict Vessels

- Destroy coastal habitats
- Hazards to navigation and shipping lanes
- Destroy sensitive marine life
- Move during storms
- Disperse oil and toxic chemicals

### NOAA's Office of Response and Restoration 

<head>
<style>
* {
  box-sizing: border-box;
}

/* Create three equal columns that floats next to each other */
.column {
  float: left;
  width: 50%;
  padding: 10px;
  
}

/* Clear floats after the columns */
.row:after {
  content: "";
  display: table;
  clear: both;
}
</style>
</head>
<body>

<div class="row" >
  <div class="column" >
    <ul align="right">
    <li> Research to address abandoned vessel issues, including legal obstacles. </li>
    <li> Working with the National Response Team to document removal efforts and related best practices.</li>
    <li> Producing guidance materials for state, territorial, and local governments.</li>
    <li> Developing databases of information on potentially polluting shipwrecks and abandoned vessels.</li>
</ul>  
  </div>
 
</div>

## Improving Shipwreck Detection using Deep Learning

### Setup the Environment

- Load the Python modules that will aid in analysis

In [ ]:
import os

In [ ]:
import arcgis
from arcgis.mapping import MapImageLayer
from arcgis.gis import GIS
from arcgis.learn import prepare_data, SingleShotDetector
from arcgis.learn import export_training_data
from arcgis.raster import ImageryLayer
from arcgis.raster.functions import apply

### Connect to the GIS

In [ ]:
gis = GIS('home')

### Examining the Source Data

NOAA provides a web service of existing ship wrecks for the coastal regions around the United States

##### Prepare the Map

In [ ]:
ms = MapImageLayer(("https://wrecks.nauticalcharts.noaa.gov/arcgis/rest"
                    "/services/public_wrecks/Wrecks_And_Obstructions/MapServer"))

In [ ]:
training_data = gis.content.get('356049f5142d45dbb9d8a681ee0d5b26')
training_data

#### Display the Training Data

In [ ]:
wreck_map = gis.map()
wreck_map.extent = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
 'xmin': -8233053.575188192,
 'ymin': 4948282.666904231,
 'xmax': -8203090.260100443,
 'ymax': 4963570.0725612445}
wreck_map.basemap = 'dark-gray-vector'
wreck_map.add_layer(ms)
wreck_map.add_layer(training_data)

In [ ]:
wreck_map.extent

In [ ]:
wreck_map

### Examine the Source Imagery


Using **Bathymetric Imagery** 

In [ ]:
imagery = apply(ImageryLayer("https://gis.ngdc.noaa.gov/arcgis/rest/services/bag_bathymetry/ImageServer"), 'ColorHillshadeBAG')

In [ ]:
imagery_map = gis.map()
imagery_map.extent = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
 'xmin': -8233053.575188192,
 'ymin': 4948282.666904231,
 'xmax': -8203090.260100443,
 'ymax': 4963570.0725612445}
imagery_map.basemap = 'dark-gray-vector'
imagery_map.add_layer(imagery)
imagery_map.add_layer(training_data)
imagery_map

## Extract Data to Chips Locally

In [ ]:
arcgis.env.verbose = True

In [ ]:
chips = export_training_data(imagery, training_data, "PNG", {"x":448,"y":448}, 
                             {"x":224,"y":224}, "PASCAL_VOC_rectangles", 
                             output_location="./data/ship_detection",
                             classvalue_field='ecode')

In [ ]:
chips

### Prepare Data

In [ ]:
path = "/arcgis/directories/rasterstore/ship_detection"

In [ ]:
data = prepare_data(path, dataset_type="PASCAL_VOC_rectangles", batch_size=32)
data

In [ ]:
data.show_batch()

## Perform Deep Learning

### Create a Single Shot Detector

In [ ]:
model = SingleShotDetector(data)
model.show_results()

#### Find Learning Rate

In [ ]:
lr = model.lr_find()
print(f"The Learning Rate is: {lr}")

In [ ]:
model.fit(10, lr)

#### Display the Results

In [ ]:
model.show_results(thresh=0.1)

#### Persist the Deep Learning Model

In [ ]:
model_package = model.save("ShipDetection")

In [ ]:
model_package = str(model_package) + "/ShipDetection.dlpk"

In [ ]:
detect_objects_model_package = gis.content.add(item_properties={"type":"Deep Learning Package",
                                                                "typeKeywords":"Deep Learning, Raster",
                                                                "title":"Shipwreck Detector Model",
                                                                "tags":"deeplearning", 'overwrite':'True'}, 
                                               data=model_package)
detect_objects_model_package

#### Install the Saved Model

In [ ]:
from arcgis.learn import Model, list_models
#detect_objects_model = Model(detect_objects_model_package)
#detect_objects_model.install()

#### Perform the Object Detection

In [ ]:
from arcgis.learn import detect_objects
out_objects = detect_objects(input_raster=imagery,
                             model=detect_objects_model,
                             output_name="ship_detection_result",
                             context={'cellSize': 0.42, 'processorType': 'GPU'},
                             gis=gis)

## Detection Results

In [ ]:
item = gis.content.get('d26702bacb644c76b8877ee2ca7273e3')
item

In [ ]:
result_map = gis.map(item)
result_map.extent = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
 'xmin': -8225394.87168831,
 'ymin': 4954360.338976395,
 'xmax': -8223322.711624816,
 'ymax': 4954838.070403137}
result_map